In [3]:
# --- Imports ---
import numpy as np
import nibabel as nib
from scipy.ndimage import distance_transform_edt, gaussian_filter
from skimage import measure

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# VTK and Open3D used in different approaches
import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

import open3d as o3d


# ---------- Parameters ----------
# NIFTI_PATH = "liver_mask_resampled_2mm.nii.gz"  # <-- update path if needed
NIFTI_PATH = '/home/ralbe/pyhppc_project/cirr_segm_clean/cirrhotic_liver_segmentation/cirr_mri_600/data/processed/cirrhotic/T1_masks/GT/351.nii.gz'
TARGET_VERTS = 2500  # approximate target vertex count
SDF_SMOOTH_SIGMA_MM = 1.0  # Gaussian smoothing strength on SDF (mm)
SAVE_MESHES = True  # set False if you don't want .ply outputs


# ---------- Loading mask & spacing ----------
def load_nifti_mask(path):
    nii = nib.load(path)
    data = nii.get_fdata()
    mask = (data > 0).astype(np.uint8)  # robust binarization
    # spacing from header (pixdim[1:4]); fallback to 2.0 mm if missing
    hdr = nii.header
    pixdim = hdr.get_zooms()[:3]
    spacing = tuple(float(p) if p > 0 else 2.0 for p in pixdim)
    return mask, spacing

mask, spacing = load_nifti_mask(NIFTI_PATH)
print("Mask shape:", mask.shape, "Spacing (mm):", spacing)

# Sanity: you said it's 2 mm isotropic, so this should print (2.0, 2.0, 2.0) or close.

Mask shape: (192, 192, 192) Spacing (mm): (2.0000617504119873, 2.0, 2.00063157081604)


In [4]:
# -------- Signed distance (in physical units mm) --------
def compute_sdf_mm(mask, spacing):
    """
    Returns signed distance field in millimetres: positive inside, negative outside.
    Uses Euclidean DT on mask and inverse mask with 'sampling' = spacing.
    """
    # Inside distance (to background)
    dt_in = distance_transform_edt(mask, sampling=spacing)
    # Outside distance (to foreground)
    dt_out = distance_transform_edt(1 - mask, sampling=spacing)
    # Signed: positive inside, negative outside
    sdf = dt_in - dt_out
    return sdf

# -------- Marching cubes utils --------
def mc_from_volume_iso(volume, spacing, level=0.0):
    """
    volume: 3D np array (e.g., SDF or mask), spacing=(sx,sy,sz)
    Returns V, F where V are in (x,y,z) coordinates in mm.
    """
    # skimage marching_cubes expects volume in (z, y, x) indexing (which numpy has),
    # and spacing as (sz, sy, sx). We'll pass accordingly, then reorder to (x,y,z).
    verts, faces, normals, values = measure.marching_cubes(
        volume, level=level, spacing=spacing[::-1]  # (sz, sy, sx)
    )
    # skimage returns vertices as (z_mm, y_mm, x_mm). Reorder to (x,y,z)
    V = verts[:, ::-1]
    F = faces.astype(np.int32)
    return V, F

# -------- Wireframe for Plotly --------
def mesh_edges_wireframe(V, F):
    """
    Compute unique edges and return x,y,z lists with None separators for Plotly Scatter3d lines.
    """
    edges = set()
    for tri in F:
        i, j, k = int(tri[0]), int(tri[1]), int(tri[2])
        for a, b in [(i, j), (j, k), (k, i)]:
            if a > b:
                a, b = b, a
            edges.add((a, b))
    x, y, z = [], [], []
    for a, b in edges:
        xa, ya, za = V[a]
        xb, yb, zb = V[b]
        x += [xa, xb, None]
        y += [ya, yb, None]
        z += [za, zb, None]
    return x, y, z

# -------- Conversion: numpy <-> VTK PolyData --------
def numpy_to_vtk_polydata(V, F):
    points = vtk.vtkPoints()
    points.SetData(numpy_to_vtk(V.astype(np.float64), deep=True))
    polys = vtk.vtkCellArray()
    # Ensure triangles:
    F = F.astype(np.int64)
    cells = vtk.vtkIdTypeArray()
    cells.SetNumberOfComponents(4)
    cells.SetNumberOfTuples(F.shape[0])
    flat = np.c_[np.full((F.shape[0], 1), 3, dtype=np.int64), F].ravel()
    for i in range(flat.size):
        cells.InsertNextValue(int(flat[i]))
    polys.SetCells(F.shape[0], cells)
    poly = vtk.vtkPolyData()
    poly.SetPoints(points)
    poly.SetPolys(polys)
    return poly

def vtk_polydata_to_numpy(poly):
    pts = vtk_to_numpy(poly.GetPoints().GetData()).astype(np.float64)
    polys = poly.GetPolys()
    polys_np = vtk_to_numpy(polys.GetData()).astype(np.int64)
    # polys_np is [n, i0, i1, i2, n, i0, ...] where n=3 for triangles
    F = polys_np.reshape(-1, 4)[:, 1:4].astype(np.int32)
    return pts, F

# -------- VTK decimation helper to target vertices --------
def vtk_decimate_to_target(poly, target_n_verts):
    nverts = poly.GetNumberOfPoints()
    if nverts <= target_n_verts:
        return poly
    reduction = max(0.0, min(0.95, 1.0 - float(target_n_verts) / float(nverts)))
    decim = vtk.vtkQuadricDecimation()
    decim.SetInputData(poly)
    decim.SetTargetReduction(reduction)
    decim.VolumePreservationOn()
    decim.Update()
    return decim.GetOutput()

# -------- Open3D decimation helper to target vertices --------
def o3d_decimate_to_target(V, F, target_n_verts):
    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(V),
        triangles=o3d.utility.Vector3iVector(F)
    )
    mesh.compute_vertex_normals()
    if np.asarray(mesh.vertices).shape[0] > target_n_verts:
        # estimate target triangles ~ 2 * target_verts as a heuristic (planar)
        target_tris = max(100, int(target_n_verts * 2))
        mesh = mesh.simplify_quadric_decimation(target_tris)
        mesh.remove_duplicated_vertices()
        mesh.remove_degenerate_triangles()
        mesh.remove_duplicated_triangles()
        mesh.remove_non_manifold_edges()
    Vd = np.asarray(mesh.vertices).copy()
    Fd = np.asarray(mesh.triangles).astype(np.int32).copy()
    return Vd, Fd

In [5]:
# --- Baseline marching cubes (binary mask) ---
V_mc_raw, F_mc_raw = mc_from_volume_iso(mask.astype(np.float32), spacing, level=0.5)
V_mc_raw_d, F_mc_raw_d = o3d_decimate_to_target(V_mc_raw, F_mc_raw, TARGET_VERTS)

print("Baseline MC raw V:", V_mc_raw.shape[0], "Decimated V:", V_mc_raw_d.shape[0])

Baseline MC raw V: 48530 Decimated V: 2504


In [6]:
def approach1_sdf_gaussian_mc(mask, spacing, target_verts, sigma_mm=1.0, save_path=None):
    # 1) Signed distance in mm
    sdf = compute_sdf_mm(mask, spacing)
    # 2) Smooth SDF (Gaussian); sigma in voxels = sigma_mm / spacing
    sx, sy, sz = spacing
    sig_vox = (sigma_mm / sx, sigma_mm / sy, sigma_mm / sz)
    sdf_s = gaussian_filter(sdf, sigma=sig_vox, mode='nearest')
    # 3) Extract zero level
    V, F = mc_from_volume_iso(sdf_s, spacing, level=0.0)
    # 4) Decimate to target verts
    Vd, Fd = o3d_decimate_to_target(V, F, target_verts)
    # 5) Save
    if save_path:
        mesh = o3d.geometry.TriangleMesh(
            vertices=o3d.utility.Vector3dVector(Vd),
            triangles=o3d.utility.Vector3iVector(Fd)
        )
        o3d.io.write_triangle_mesh(save_path, mesh, print_progress=False)
    return Vd, Fd

V_A, F_A = approach1_sdf_gaussian_mc(
    mask, spacing, TARGET_VERTS, sigma_mm=SDF_SMOOTH_SIGMA_MM,
    save_path="mesh_approach1_sdf_mc.ply" if SAVE_MESHES else None
)
print("Approach 1 vertices:", V_A.shape[0])


Approach 1 vertices: 2506


In [7]:
def approach2_vtk_flying_edges_taubin(mask, spacing, target_verts,
                                      taubin_passband=0.12, taubin_iters=25, save_path=None):
    # Create vtkImageData from numpy mask
    img = vtk.vtkImageData()
    nz, ny, nx = mask.shape  # note: numpy order is z,y,x
    img.SetDimensions(nx, ny, nz)
    img.SetSpacing(spacing[0], spacing[1], spacing[2])
    # Origin at (0,0,0). If you need world coordinates, set it here.
    flat = np.ascontiguousarray(mask.transpose(2,1,0).ravel(order='C')).astype(np.uint8)
    vtk_array = numpy_to_vtk(flat, deep=True, array_type=vtk.VTK_UNSIGNED_CHAR)
    vtk_array.SetName("mask")
    img.GetPointData().SetScalars(vtk_array)

    # Flying Edges isosurface at 0.5
    fe = vtk.vtkFlyingEdges3D()
    fe.SetInputData(img)
    fe.SetValue(0, 0.5)
    fe.Update()
    poly = fe.GetOutput()

    # Taubin (windowed sinc) smoothing - volume-preserving
    ws = vtk.vtkWindowedSincPolyDataFilter()
    ws.SetInputData(poly)
    ws.SetNumberOfIterations(taubin_iters)
    ws.SetPassBand(taubin_passband)    # lower = more smoothing (0.05–0.2 good range)
    ws.BoundarySmoothingOff()
    ws.FeatureEdgeSmoothingOff()
    ws.NonManifoldSmoothingOn()
    ws.NormalizeCoordinatesOn()
    ws.Update()
    smooth_poly = ws.GetOutput()

    # Decimate to target vertices
    dec_poly = vtk_decimate_to_target(smooth_poly, target_verts)

    # Convert to numpy
    V, F = vtk_polydata_to_numpy(dec_poly)

    # Save
    if save_path:
        writer = vtk.vtkPLYWriter()
        writer.SetFileName(save_path)
        writer.SetInputData(dec_poly)
        writer.SetFileTypeToASCII()
        writer.Write()
    return V, F

V_B, F_B = approach2_vtk_flying_edges_taubin(
    mask, spacing, TARGET_VERTS, taubin_passband=0.12, taubin_iters=25,
    save_path="mesh_approach2_vtk_taubin.ply" if SAVE_MESHES else None
)
print("Approach 2 vertices:", V_B.shape[0])

Approach 2 vertices: 2505


In [8]:
def approach3_poisson_from_sdf(mask, spacing, target_verts,
                               sdf_sigma_mm=1.0, poisson_depth=8, save_path=None):
    # 1) SDF + (optional) light smoothing for clean normals
    sdf = compute_sdf_mm(mask, spacing)
    sx, sy, sz = spacing
    sig_vox = (sdf_sigma_mm / sx, sdf_sigma_mm / sy, sdf_sigma_mm / sz)
    sdf_s = gaussian_filter(sdf, sigma=sig_vox, mode='nearest')

    # 2) Extract zero level-set (marching cubes) to get surface samples
    V, F = mc_from_volume_iso(sdf_s, spacing, level=0.0)

    # 3) Estimate vertex normals from SDF gradient (central differences)
    # Build a trilinear gradient on the grid and sample at vertex coordinates.
    # For simplicity, we approximate by nearest voxel gradient.
    # Compute gradient in mm units (sdf already in mm, so divide by mm spacing to get dimensionless curvature)
    gy, gx, gz = np.gradient(sdf_s, spacing[1], spacing[0], spacing[2])  # gradient w.r.t y,x,z axes in mm
    # Map V (x,y,z) to voxel indices (approx by dividing by spacing)
    ix = np.clip(np.round(V[:, 0] / spacing[0]).astype(int), 0, sdf_s.shape[2]-1)
    iy = np.clip(np.round(V[:, 1] / spacing[1]).astype(int), 0, sdf_s.shape[1]-1)
    iz = np.clip(np.round(V[:, 2] / spacing[2]).astype(int), 0, sdf_s.shape[0]-1)
    # gradients were computed on (z,y,x)-ordered array; indexes (iz,iy,ix)
    G = np.stack([gx[iz, iy, ix], gy[iz, iy, ix], gz[iz, iy, ix]], axis=1)
    # Normals point outward for negative gradients of SDF (since inside positive => outward normal ~ +grad)
    # Open3D expects normals pointing OUTWARD; we normalize:
    norms = G
    n_norm = np.linalg.norm(norms, axis=1, keepdims=True) + 1e-12
    normals = norms / n_norm

    # 4) Create point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(V)
    pcd.normals = o3d.utility.Vector3dVector(normals)

    # 5) Poisson reconstruction
    mesh_poisson, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=poisson_depth
    )

    # 6) Crop mesh to inside bounding box of points (remove far-field surfaces)
    pts = np.asarray(pcd.points)
    min_bb = pts.min(axis=0) - 5.0
    max_bb = pts.max(axis=0) + 5.0
    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bb, max_bb)
    mesh_poisson = mesh_poisson.crop(bbox)

    # Clean up
    mesh_poisson.remove_duplicated_vertices()
    mesh_poisson.remove_degenerate_triangles()
    mesh_poisson.remove_duplicated_triangles()
    mesh_poisson.remove_non_manifold_edges()

    # 7) Decimate to target vertices
    Vp = np.asarray(mesh_poisson.vertices)
    Fp = np.asarray(mesh_poisson.triangles).astype(np.int32)
    Vd, Fd = o3d_decimate_to_target(Vp, Fp, target_verts)

    # 8) Save
    if save_path:
        mesh_out = o3d.geometry.TriangleMesh(
            vertices=o3d.utility.Vector3dVector(Vd),
            triangles=o3d.utility.Vector3iVector(Fd)
        )
        o3d.io.write_triangle_mesh(save_path, mesh_out, print_progress=False)

    return Vd, Fd

V_C, F_C = approach3_poisson_from_sdf(
    mask, spacing, TARGET_VERTS, sdf_sigma_mm=0.8, poisson_depth=8,
    save_path="mesh_approach3_poisson.ply" if SAVE_MESHES else None
)


In [9]:
print("Approach 3 vertices:", V_C.shape[0])

Approach 3 vertices: 2317


In [10]:
# # Prepare wireframes
# def wireframe_trace(V, F, name, color="black", width=1):
#     x, y, z = mesh_edges_wireframe(V, F)
#     return go.Scatter3d(
#         x=x, y=y, z=z,
#         mode="lines",
#         line=dict(color=color, width=width),
#         name=name,
#         showlegend=True
#     )

# # Mask isosurface visualization
# def mask_isosurface_trace(mask, spacing, name="Mask Isosurface", color="rgba(200,30,30,0.5)"):
#     # Plotly Isosurface expects value on 3D grid and grid coordinates; we can pass as is.
#     # Coordinates: create regular grid in mm
#     nz, ny, nx = mask.shape
#     x = np.arange(nx) * spacing[0]
#     y = np.arange(ny) * spacing[1]
#     z = np.arange(nz) * spacing[2]
#     # Flatten to supply to plotly
#     X, Y, Z = np.meshgrid(x, y, z, indexing='xy')
#     vals = mask.transpose(2,1,0)  # -> (x,y,z) for flatten order='F' below

#     return go.Isosurface(
#         x=X.flatten(order='F'), y=Y.flatten(order='F'), z=Z.flatten(order='F'),
#         value=vals.flatten(order='F'),
#         isomin=0.5, isomax=0.5, surface_count=1,
#         opacity=0.4, showscale=False,
#         caps=dict(x_show=False, y_show=False, z_show=False),
#         colorscale=[[0, color], [1, color]],
#         name=name
#     )

# # Build figure with 1x5 subplots
# fig = make_subplots(
#     rows=1, cols=5, specs=[[{'type':'scene'}]*5],
#     subplot_titles=(
#         "Mask (isosurface)", 
#         "Baseline MC (wireframe)",
#         "Approach 1: SDF+MC",
#         "Approach 2: VTK+Taubin",
#         "Approach 3: Poisson"
#     )
# )

# # 1) Mask isosurface
# fig.add_trace(mask_isosurface_trace(mask, spacing, color="rgba(200,50,50,0.5)"), row=1, col=1)

# # 2) Baseline MC wireframe
# fig.add_trace(wireframe_trace(V_mc_raw_d, F_mc_raw_d, "Baseline MC", color="#444", width=1), row=1, col=2)

# # 3) Approach 1
# fig.add_trace(wireframe_trace(V_A, F_A, "A: SDF+MC", color="#1f77b4", width=1), row=1, col=3)

# # 4) Approach 2
# fig.add_trace(wireframe_trace(V_B, F_B, "B: VTK+Taubin", color="#2ca02c", width=1), row=1, col=4)

# # 5) Approach 3
# fig.add_trace(wireframe_trace(V_C, F_C, "C: Poisson", color="#d62728", width=1), row=1, col=5)

# # Shared scene settings
# for i in range(1, 6):
#     scene_id = f"scene{i}" if i > 1 else "scene"
#     fig.layout[scene_id].update(
#         xaxis=dict(title="X (mm)", showgrid=False, zeroline=False),
#         yaxis=dict(title="Y (mm)", showgrid=False, zeroline=False),
#         zaxis=dict(title="Z (mm)", showgrid=False, zeroline=False),
#         aspectmode="data"
#     )

# fig.update_layout(height=600, width=2200, title_text="Liver Mask Meshing — Baseline vs Three Smoothing Pipelines")


In [11]:
import trimesh

mesh_A = trimesh.Trimesh(V_A, F_A)
mesh_B = trimesh.Trimesh(V_B, F_B)
mesh_C = trimesh.Trimesh(V_C, F_C)
# Save all 3 meshes to OBJ files
mesh_A.export("mesh_A.obj")
mesh_B.export("mesh_B.obj")
mesh_C.export("mesh_C.obj")

'# https://github.com/mikedh/trimesh\nv 224.90787840 243.28119671 177.38092685\nv 212.85913344 118.58131786 255.45061705\nv 288.47131392 129.46098983 213.02171403\nv 162.25163008 123.49713702 143.14613260\nv 222.31867136 237.60214110 187.70585775\nv 207.47683328 169.39471371 255.70254101\nv 227.94644480 83.63609802 191.95084601\nv 300.99683840 238.51087601 146.92142641\nv 234.30282752 139.33341970 244.45472265\nv 244.44720384 101.47227155 154.61334358\nv 188.88251136 105.78116982 233.85109011\nv 173.30870017 109.86409400 135.06616886\nv 270.36370689 205.00343707 249.29746138\nv 239.71908353 276.26520933 118.56667864\nv 192.89632257 164.89799891 150.99840501\nv 189.87840513 172.36565399 175.61702198\nv 210.92799489 226.12416213 199.39666428\nv 180.82216705 221.21019076 129.31976972\nv 183.41478657 212.52264607 161.61124061\nv 183.06770946 155.29859197 217.58622913\nv 189.08394754 216.52410228 186.64787750\nv 308.00763394 228.33912395 151.40339889\nv 180.95758338 158.75002164 182.8890832

In [12]:
print('Bingo :) ')

Bingo :) 


In [13]:
import numpy as np
import nibabel as nib
from scipy.ndimage import distance_transform_edt, gaussian_filter, map_coordinates
from skimage import measure
import open3d as o3d
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ---------------- Parameters ----------------
NIFTI_PATH = '/home/ralbe/pyhppc_project/cirr_segm_clean/cirrhotic_liver_segmentation/cirr_mri_600/data/processed/cirrhotic/T1_masks/GT/351.nii.gz'

TARGET_VERTS = 2500
SDF_SMOOTH_SIGMA_MM = 0.8   # start conservative; 0.6–1.2 usually ok
POISSON_DEPTH = 8           # 7–9 good for organs
SAVE_MESHES = False         # set True to write meshes
OUT_PREFIX = "liver_mesh"

# ---------------- Load NIfTI ----------------
def load_nifti_mask_xyz(path):
    nii = nib.load(path)
    data = nii.get_fdata()
    mask = (data > 0).astype(np.uint8)  # shape = (X, Y, Z)
    dx, dy, dz = nii.header.get_zooms()[:3]
    spacing_xyz = (float(dx), float(dy), float(dz))
    return mask, spacing_xyz, nii

mask_xyz, spacing_xyz, nii = load_nifti_mask_xyz(NIFTI_PATH)
dx, dy, dz = spacing_xyz
print(f"mask shape (X,Y,Z): {mask_xyz.shape} | spacing (dx,dy,dz): {spacing_xyz}")

# Quick checks
assert mask_xyz.ndim == 3, "Mask must be 3D"
assert mask_xyz.sum() > 0, "Mask appears empty"


mask shape (X,Y,Z): (192, 192, 192) | spacing (dx,dy,dz): (2.0000617504119873, 2.0, 2.00063157081604)


In [14]:
# ---- Marching cubes on XYZ volume with correct spacing ----
def mc_from_xyz(volume_xyz, spacing_xyz, level):
    """
    volume_xyz: ndarray (X,Y,Z) with physical spacing (dx,dy,dz).
    skimage.measure.marching_cubes expects spacing to match the order of axes.
    It returns vertices in the SAME axis order as input (axis 0 -> coordinate 0).
    So with (X,Y,Z) input and spacing=(dx,dy,dz), returned verts are (x_mm, y_mm, z_mm).
    """
    verts, faces, normals, values = measure.marching_cubes(
        volume_xyz, level=level, spacing=spacing_xyz  # NOTE: no reversing!
    )
    V = verts.astype(np.float64)
    F = faces.astype(np.int32)
    return V, F

# ---- SDF in mm for XYZ-arrays ----
def compute_sdf_mm_xyz(mask_xyz, spacing_xyz):
    # sampling must match axis order of the array (X,Y,Z) -> (dx,dy,dz)
    dt_in = distance_transform_edt(mask_xyz, sampling=spacing_xyz)
    dt_out = distance_transform_edt(1 - mask_xyz, sampling=spacing_xyz)
    return dt_in - dt_out  # positive inside

# ---- Open3D decimation to ~target verts ----
def decimate_o3d_to_target(V, F, target_verts):
    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(V),
        triangles=o3d.utility.Vector3iVector(F)
    )
    mesh.compute_vertex_normals()
    n0 = np.asarray(mesh.vertices).shape[0]
    if n0 > target_verts:
        # Aim triangles ≈ 2 * verts (heuristic), clamp min
        target_tris = max(500, int(target_verts * 2))
        mesh = mesh.simplify_quadric_decimation(target_tris)
    mesh.remove_duplicated_vertices()
    mesh.remove_degenerate_triangles()
    mesh.remove_duplicated_triangles()
    mesh.remove_non_manifold_edges()
    Vd = np.asarray(mesh.vertices).copy()
    Fd = np.asarray(mesh.triangles).astype(np.int32).copy()
    return Vd, Fd

# ---- Wireframe for Plotly ----
def mesh_edges_wireframe(V, F):
    edges = set()
    for tri in F:
        i, j, k = int(tri[0]), int(tri[1]), int(tri[2])
        for a, b in [(i, j), (j, k), (k, i)]:
            if a > b: a, b = b, a
            edges.add((a, b))
    x, y, z = [], [], []
    for a, b in edges:
        xa, ya, za = V[a]
        xb, yb, zb = V[b]
        x += [xa, xb, None]
        y += [ya, yb, None]
        z += [za, zb, None]
    return x, y, z

In [15]:
V_mc_raw, F_mc_raw = mc_from_xyz(mask_xyz.astype(np.float32), spacing_xyz, level=0.5)
V_mc_raw_d, F_mc_raw_d = decimate_o3d_to_target(V_mc_raw, F_mc_raw, TARGET_VERTS)
print("Baseline MC verts:", V_mc_raw.shape[0], "→ decimated:", V_mc_raw_d.shape[0])

Baseline MC verts: 48530 → decimated: 2504


In [16]:
def approach1_sdf_gaussian_mc_xyz(mask_xyz, spacing_xyz, target_verts, sigma_mm=0.8, save_path=None):
    dx, dy, dz = spacing_xyz
    sdf = compute_sdf_mm_xyz(mask_xyz, spacing_xyz)
    # Gaussian sigma in voxels per axis:
    sig_vox = (sigma_mm / dx, sigma_mm / dy, sigma_mm / dz)
    sdf_s = gaussian_filter(sdf, sigma=sig_vox, mode='nearest')
    V, F = mc_from_xyz(sdf_s, spacing_xyz, level=0.0)
    Vd, Fd = decimate_o3d_to_target(V, F, target_verts)
    if save_path:
        m = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(Vd), o3d.utility.Vector3iVector(Fd))
        o3d.io.write_triangle_mesh(save_path, m, print_progress=False)
    return Vd, Fd

V_A, F_A = approach1_sdf_gaussian_mc_xyz(
    mask_xyz, spacing_xyz, TARGET_VERTS, sigma_mm=SDF_SMOOTH_SIGMA_MM,
    save_path=f"{OUT_PREFIX}_A_sdf_mc.ply" if SAVE_MESHES else None
)
print("Approach 1 verts:", V_A.shape[0])

Approach 1 verts: 2508


In [17]:
def approach2_taubin_open3d_xyz(mask_xyz, spacing_xyz, target_verts, iters=20, save_path=None):
    # Extract binary isosurface then apply Taubin smoothing (volume-friendly)
    V, F = mc_from_xyz(mask_xyz.astype(np.float32), spacing_xyz, level=0.5)
    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(V),
        triangles=o3d.utility.Vector3iVector(F)
    )
    mesh = mesh.filter_smooth_taubin(number_of_iterations=iters)
    mesh.compute_vertex_normals()
    Vt = np.asarray(mesh.vertices)
    Ft = np.asarray(mesh.triangles).astype(np.int32)
    Vd, Fd = decimate_o3d_to_target(Vt, Ft, target_verts)
    if save_path:
        m = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(Vd), o3d.utility.Vector3iVector(Fd))
        o3d.io.write_triangle_mesh(save_path, m, print_progress=False)
    return Vd, Fd

V_B, F_B = approach2_taubin_open3d_xyz(
    mask_xyz, spacing_xyz, TARGET_VERTS, iters=20,
    save_path=f"{OUT_PREFIX}_B_taubin.ply" if SAVE_MESHES else None
)
print("Approach 2 verts:", V_B.shape[0])

Approach 2 verts: 2504


In [18]:
def approach3_poisson_from_sdf_xyz(mask_xyz, spacing_xyz, target_verts, sdf_sigma_mm=0.6, poisson_depth=8, save_path=None):
    dx, dy, dz = spacing_xyz
    sdf = compute_sdf_mm_xyz(mask_xyz, spacing_xyz)
    sig_vox = (sdf_sigma_mm / dx, sdf_sigma_mm / dy, sdf_sigma_mm / dz)
    sdf_s = gaussian_filter(sdf, sigma=sig_vox, mode='nearest')

    # Get zero-level samples
    V, F = mc_from_xyz(sdf_s, spacing_xyz, level=0.0)

    # Compute gradient in mm-aware coordinates (gx=∂/∂x, gy=∂/∂y, gz=∂/∂z)
    gx, gy, gz = np.gradient(sdf_s, dx, dy, dz)

    # Trilinear sample gradients at vertex locations (indices in voxel coords)
    coords = np.vstack((V[:,0] / dx, V[:,1] / dy, V[:,2] / dz))
    gx_s = map_coordinates(gx, coords, order=1, mode='nearest')
    gy_s = map_coordinates(gy, coords, order=1, mode='nearest')
    gz_s = map_coordinates(gz, coords, order=1, mode='nearest')

    G = np.stack([gx_s, gy_s, gz_s], axis=1)
    nrm = np.linalg.norm(G, axis=1, keepdims=True) + 1e-12
    normals = G / nrm  # outward for positive-inside SDF ~ +grad

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(V)
    pcd.normals = o3d.utility.Vector3dVector(normals)

    mesh_p, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=poisson_depth)

    # Crop to the point cloud bounds (+ margin mm)
    pts = np.asarray(pcd.points)
    bb_min = pts.min(axis=0) - 5.0
    bb_max = pts.max(axis=0) + 5.0
    bbox = o3d.geometry.AxisAlignedBoundingBox(bb_min, bb_max)
    mesh_p = mesh_p.crop(bbox)

    mesh_p.remove_duplicated_vertices()
    mesh_p.remove_degenerate_triangles()
    mesh_p.remove_duplicated_triangles()
    mesh_p.remove_non_manifold_edges()

    Vp = np.asarray(mesh_p.vertices)
    Fp = np.asarray(mesh_p.triangles).astype(np.int32)
    Vd, Fd = decimate_o3d_to_target(Vp, Fp, target_verts)

    if save_path:
        m = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(Vd), o3d.utility.Vector3iVector(Fd))
        o3d.io.write_triangle_mesh(save_path, m, print_progress=False)
    return Vd, Fd

V_C, F_C = approach3_poisson_from_sdf_xyz(
    mask_xyz, spacing_xyz, TARGET_VERTS, sdf_sigma_mm=0.6, poisson_depth=POISSON_DEPTH,
    save_path=f"{OUT_PREFIX}_C_poisson.ply" if SAVE_MESHES else None
)
print("Approach 3 verts:", V_C.shape[0])

Approach 3 verts: 2531


In [19]:
print("Affine:\n", nii.affine)
print("Unique values in mask:", np.unique(mask_xyz))
print("Mask voxel count:", mask_xyz.sum(), " | volume (mL) ≈", mask_xyz.sum() * dx*dy*dz / 1000)

# Quick check: compare bbox extents in mm vs expected liver size (~150-250 mm)
nz = mask_xyz.shape[2]
extents_mm = (mask_xyz.shape[0]*dx, mask_xyz.shape[1]*dy, mask_xyz.shape[2]*dz)


Affine:
 [[-2.00000000e+00 -1.11167049e-06 -5.02592064e-02  1.84315353e+02]
 [-1.11167049e-06 -2.00000000e+00  5.09588339e-04  1.38558899e+02]
 [-1.57060437e-02  1.59246760e-04  2.00000000e+00 -2.18477325e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Unique values in mask: [0 1]
Mask voxel count: 330839  | volume (mL) ≈ 2647.6295366123195


In [20]:
import trimesh 


mesh_A = trimesh.Trimesh(V_A, F_A)
mesh_B = trimesh.Trimesh(V_B, F_B)
mesh_C = trimesh.Trimesh(V_C, F_C)

OUT_PREFIX = "liver_mesh"
mesh_A.export(f"{OUT_PREFIX}_A_trimesh.obj")
mesh_B.export(f"{OUT_PREFIX}_B_trimesh.obj")
mesh_C.export(f"{OUT_PREFIX}_C_trimesh.obj")

'# https://github.com/mikedh/trimesh\nv 166.38953472 250.95867397 233.74630668\nv 81.08530944 151.12462039 179.46278451\nv 76.80187904 146.19736567 213.70406563\nv 191.19630848 225.25029185 299.57465711\nv 250.97629696 175.24311185 199.80081034\nv 78.92010240 174.92927033 265.48080008\nv 205.83908864 234.39503057 278.15362101\nv 247.95843840 142.83258014 240.50330088\nv 159.99218944 100.40180153 242.28187848\nv 162.90136832 113.91769827 262.73481061\nv 97.61582080 130.87916516 239.73164333\nv 72.85243136 191.61470652 272.27570163\nv 96.45667841 192.97578281 171.54005566\nv 217.82279937 149.34570804 194.93711112\nv 176.99453953 96.01000844 246.77984597\nv 88.27324673 196.91525935 287.18394083\nv 170.70774529 241.67290161 226.72396930\nv 177.62443777 250.27012052 250.53688109\nv 231.88435201 132.26611789 291.75078261\nv 198.26717185 127.23755173 286.19114285\nv 82.75412481 180.01199232 270.74761922\nv 215.84014081 207.56745204 299.83577303\nv 142.88683522 192.21862638 194.30647979\nv 253

In [21]:
import numpy as np
from scipy.ndimage import distance_transform_edt, gaussian_filter, zoom, map_coordinates
from skimage.restoration import denoise_tv_chambolle
from skimage import measure
import open3d as o3d

# ---- Compute SDF (mm) on current grid ----
def compute_sdf_mm(mask, spacing):
    dt_in  = distance_transform_edt(mask,         sampling=spacing)
    dt_out = distance_transform_edt(1 - mask,     sampling=spacing)
    return dt_in - dt_out  # +inside / -outside, in millimetres

# ---- Supersample SDF to a finer isotropic grid (e.g., 1.0 mm) ----
def upsample_sdf(sdf, spacing, new_iso=1.0):
    sz, sy, sx = spacing[2], spacing[1], spacing[0]  # careful with array order (z,y,x)
    zoom_factors = (sz / new_iso, sy / new_iso, sx / new_iso)
    sdf_up = zoom(sdf, zoom=zoom_factors, order=1)  # trilinear
    new_spacing = (new_iso, new_iso, new_iso)
    return sdf_up, new_spacing

# ---- Volume of mask (mm^3) ----
def voxel_volume_mm3(spacing):
    return float(spacing[0] * spacing[1] * spacing[2])

def mask_volume_mm3(mask, spacing):
    return float(mask.sum()) * voxel_volume_mm3(spacing)

# ---- Fast volume from SDF threshold at 'level' (approx) ----
def volume_from_sdf_level(sdf, spacing, level):
    # Approximate by voxel counting (good enough for iso-level search)
    vox_vol = voxel_volume_mm3(spacing)
    return float((sdf >= level).sum()) * vox_vol

# ---- Find iso-level that preserves original volume (bisection) ----
def find_volume_matched_isolevel(sdf, spacing, target_volume_mm3, 
                                 lo=None, hi=None, tol_rel=0.01, max_iter=40):
    # Reasonable bracket around 0 in mm (SDF values are distances)
    if lo is None:
        lo = -5.0 * spacing[0]  # inside expands
    if hi is None:
        hi = +5.0 * spacing[0]  # inside shrinks
    # Ensure monotonic ordering: volume decreases with higher level
    vol_lo = volume_from_sdf_level(sdf, spacing, lo)
    vol_hi = volume_from_sdf_level(sdf, spacing, hi)
    if not (vol_lo >= target_volume_mm3 >= vol_hi):
        # widen bracket if needed
        minv = float(sdf.min())
        maxv = float(sdf.max())
        lo = max(minv, lo - 10.0)
        hi = min(maxv, hi + 10.0)
        vol_lo = volume_from_sdf_level(sdf, spacing, lo)
        vol_hi = volume_from_sdf_level(sdf, spacing, hi)
        if not (vol_lo >= target_volume_mm3 >= vol_hi):
            # fallback to 0.0 if we can't bracket (still works)
            return 0.0

    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        vol_mid = volume_from_sdf_level(sdf, spacing, mid)
        err = (vol_mid - target_volume_mm3) / target_volume_mm3
        if abs(err) <= tol_rel:
            return mid
        if vol_mid > target_volume_mm3:
            # too big → raise level (shrink)
            lo = mid
        else:
            hi = mid
    return 0.5 * (lo + hi)

# ---- Marching cubes on any scalar field with correct spacing ----
def mc_from_volume_iso(volume, spacing, level):
    verts, faces, _, _ = measure.marching_cubes(
        volume, level=level, spacing=spacing[::-1]  # (sz, sy, sx)
    )
    V = verts[:, ::-1]               # (x,y,z)
    F = faces.astype(np.int32)
    return V, F

# ---- Open3D decimate + clean to target vertex count ----
def o3d_decimate_to_target(V, F, target_n_verts):
    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(V),
        triangles=o3d.utility.Vector3iVector(F)
    )
    mesh.compute_vertex_normals()
    nverts = np.asarray(mesh.vertices).shape[0]
    if nverts > target_n_verts:
        target_tris = max(200, int(target_n_verts * 2))  # heuristic
        mesh = mesh.simplify_quadric_decimation(target_tris)
    mesh.remove_duplicated_vertices()
    mesh.remove_degenerate_triangles()
    mesh.remove_duplicated_triangles()
    mesh.remove_non_manifold_edges()
    return np.asarray(mesh.vertices).copy(), np.asarray(mesh.triangles).astype(np.int32).copy()

# ---- Stronger but safe mesh smoothing: Taubin + Bilateral ----
def smooth_mesh_taubin_bilateral(V, F, iters_taubin=50, iters_bilat=2, sigma_len=2.0, sigma_ang=0.15):
    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(V),
        triangles=o3d.utility.Vector3iVector(F)
    )
    mesh = mesh.filter_smooth_taubin(number_of_iterations=iters_taubin)
    for _ in range(iters_bilat):
        mesh = mesh.filter_smooth_bilateral(number_of_iterations=1,
                                            sigma_length=sigma_len,
                                            sigma_angle=sigma_ang)
    mesh.compute_vertex_normals()
    return np.asarray(mesh.vertices).copy(), np.asarray(mesh.triangles).astype(np.int32).copy()

# ---- Trilinear sampling of gradient at floating coordinates ----
def gradient_trilinear(sdf, spacing, pts_xyz):
    # Compute gradients of sdf in mm along (z,y,x) axes
    gy, gx, gz = np.gradient(sdf, spacing[1], spacing[0], spacing[2])  # beware order
    # Convert world (x,y,z) to voxel (ix,iy,iz) floats
    ix = pts_xyz[:, 0] / spacing[0]
    iy = pts_xyz[:, 1] / spacing[1]
    iz = pts_xyz[:, 2] / spacing[2]
    coords = np.vstack([iz, iy, ix])  # map_coordinates expects (z,y,x) stacks

    gx_s = map_coordinates(gx, coords, order=1, mode='nearest')
    gy_s = map_coordinates(gy, coords, order=1, mode='nearest')
    gz_s = map_coordinates(gz, coords, order=1, mode='nearest')
    G = np.stack([gx_s, gy_s, gz_s], axis=1)
    N = G / (np.linalg.norm(G, axis=1, keepdims=True) + 1e-12)
    return N

In [22]:
mask, spacing = load_nifti_mask(NIFTI_PATH)
TARGET_VERTS = 2500

In [29]:
def approach_A_plus(mask, spacing, target_verts,
                    up_iso=1.0, tv_weight=0.05, tv_iters=100,
                    gauss_sigma_mm=1.5,
                    taubin_iters=60, bilat_iters=2, bilat_sigma_len=1.5, bilat_sigma_ang=0.12):
    # 1) SDF on current grid (2 mm), in millimetres
    sdf = compute_sdf_mm(mask, spacing)

    # 2) TV-regularize the SDF (edge-preserving) + light Gaussian in mm
    # TV works in index space → convert weight for 3D array empirically
    sdf_tv = denoise_tv_chambolle(sdf, weight=tv_weight, channel_axis=None)
    sig_vox = tuple(gauss_sigma_mm / s for s in spacing)
    sdf_f = gaussian_filter(sdf_tv, sigma=sig_vox, mode='nearest')

    # 3) Super-sample the SDF to a finer isotropic grid (1.0 mm)
    sdf_up, new_spacing = upsample_sdf(sdf_f, spacing, new_iso=up_iso)

    # 4) Pick iso-level that preserves the original volume
    vol_target = mask_volume_mm3(mask, spacing)
    iso_level = find_volume_matched_isolevel(sdf_up, new_spacing, vol_target, tol_rel=0.01)

    # 5) Extract with marching cubes at that iso-level
    V, F = mc_from_volume_iso(sdf_up, new_spacing, level=iso_level)

    # 6) Smooth (Taubin + bilateral) then decimate to target verts
    # V_s, F_s = smooth_mesh_taubin(V, F,
    #                                         iters_taubin=taubin_iters,
    #                                         iters_bilat=bilat_iters,
    #                                         sigma_len=bilat_sigma_len,
    #                                         sigma_ang=bilat_sigma_ang)
    V_d, F_d = o3d_decimate_to_target(V, F, target_verts)
    return V_d, F_d, dict(iso_level=iso_level, vol_target=vol_target, spacing=new_spacing)

V_Ap, F_Ap, info_Ap = approach_A_plus(
    mask, spacing, TARGET_VERTS,
    up_iso=1.0, tv_weight=0.05, tv_iters=100, gauss_sigma_mm=1.5,
    taubin_iters=60, bilat_iters=2, bilat_sigma_len=1.5, bilat_sigma_ang=0.12
)
print("A+ verts:", V_Ap.shape[0], "iso*", info_Ap["iso_level"], "spacing used:", info_Ap["spacing"])


A+ verts: 2502 iso* 0.0 spacing used: (1.0, 1.0, 1.0)


In [30]:
def approach_B_plus(mask, spacing, target_verts,
                    up_iso=1.0, gauss_sigma_mm=2.0,
                    taubin_iters=80, bilat_iters=3, bilat_sigma_len=2.0, bilat_sigma_ang=0.15):
    sdf = compute_sdf_mm(mask, spacing)
    sig_vox = tuple(gauss_sigma_mm / s for s in spacing)
    sdf_f = gaussian_filter(sdf, sigma=sig_vox, mode='nearest')

    sdf_up, new_spacing = upsample_sdf(sdf_f, spacing, new_iso=up_iso)

    vol_target = mask_volume_mm3(mask, spacing)
    iso_level = find_volume_matched_isolevel(sdf_up, new_spacing, vol_target, tol_rel=0.01)

    V, F = mc_from_volume_iso(sdf_up, new_spacing, level=iso_level)

    # V_s, F_s = smooth_mesh_taubin_bilateral(V, F,
    #                                         iters_taubin=taubin_iters,
    #                                         iters_bilat=bilat_iters,
    #                                         sigma_len=bilat_sigma_len,
    #                                         sigma_ang=bilat_sigma_ang)
    V_d, F_d = o3d_decimate_to_target(V, F, target_verts)
    return V_d, F_d, dict(iso_level=iso_level, spacing=new_spacing)

V_Bp, F_Bp, info_Bp = approach_B_plus(
    mask, spacing, TARGET_VERTS, up_iso=1.0, gauss_sigma_mm=2.0,
    taubin_iters=80, bilat_iters=3, bilat_sigma_len=2.0, bilat_sigma_ang=0.15
)
print("B+ verts:", V_Bp.shape[0], "iso*", info_Bp["iso_level"], "spacing:", info_Bp["spacing"])

B+ verts: 2502 iso* 0.0 spacing: (1.0, 1.0, 1.0)


In [31]:
def approach_C_plus(mask, spacing, target_verts,
                    up_iso=1.0, tv_weight=0.03, gauss_sigma_mm=1.2,
                    poisson_depth=9,
                    taubin_iters=40, bilat_iters=2, bilat_sigma_len=1.5, bilat_sigma_ang=0.12):
    # 1) SDF → (optional) light TV → Gaussian
    sdf = compute_sdf_mm(mask, spacing)
    sdf_tv = denoise_tv_chambolle(sdf, weight=tv_weight, channel_axis=None, max_num_iter=80)
    sig_vox = tuple(gauss_sigma_mm / s for s in spacing)
    sdf_f = gaussian_filter(sdf_tv, sigma=sig_vox, mode='nearest')

    # 2) Up-sample SDF for denser sampling
    sdf_up, new_spacing = upsample_sdf(sdf_f, spacing, new_iso=up_iso)

    # 3) Extract zero level to sample points
    V0, F0 = mc_from_volume_iso(sdf_up, new_spacing, level=0.0)

    # 4) Trilinear normals from SDF gradient
    N0 = gradient_trilinear(sdf_up, new_spacing, V0)

    # 5) Poisson reconstruction, then trim to bbox of points
    pcd = o3d.geometry.PointCloud()
    pcd.points  = o3d.utility.Vector3dVector(V0)
    pcd.normals = o3d.utility.Vector3dVector(N0)
    mesh_p, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=poisson_depth)

    pts = np.asarray(pcd.points)
    bbox = o3d.geometry.AxisAlignedBoundingBox(pts.min(0) - 5.0, pts.max(0) + 5.0)
    mesh_p = mesh_p.crop(bbox)
    mesh_p.remove_duplicated_vertices()
    mesh_p.remove_degenerate_triangles()
    mesh_p.remove_duplicated_triangles()
    mesh_p.remove_non_manifold_edges()

    Vp = np.asarray(mesh_p.vertices); Fp = np.asarray(mesh_p.triangles).astype(np.int32)

    # 6) Smooth (Taubin + bilateral) & decimate
    # V_s, F_s = smooth_mesh_taubin_bilateral(Vp, Fp,
    #                                         iters_taubin=taubin_iters,
    #                                         iters_bilat=bilat_iters,
    #                                         sigma_len=bilat_sigma_len,
    #                                         sigma_ang=bilat_sigma_ang)
    V_d, F_d = o3d_decimate_to_target(Vp, Fp, target_verts)
    return V_d, F_d, dict(spacing=new_spacing)

V_Cp, F_Cp, info_Cp = approach_C_plus(
    mask, spacing, TARGET_VERTS,
    up_iso=1.0, tv_weight=0.03, gauss_sigma_mm=1.2, poisson_depth=9,
    taubin_iters=40, bilat_iters=2, bilat_sigma_len=1.5, bilat_sigma_ang=0.12
)
print("C+ verts:", V_Cp.shape[0], "spacing:", info_Cp["spacing"])

TypeError: denoise_tv_chambolle() got an unexpected keyword argument 'max_num_iter'

In [33]:
import trimesh



# mesh_A = trimesh.Trimesh(V_Ap,F_ap)
mesh_B = trimesh.Trimesh(V_Bp, F_Bp)
# mesh_C = trimesh.Trimesh(V_Cp, F_Cp)

OUT_PREFIX = "liver_mesh_third_and_last_attempt"
# mesh_A.export(f"{OUT_PREFIX}_A_trimesh.obj")
mesh_B.export(f"{OUT_PREFIX}_B_trimesh.obj")
# mesh_C.export(f"{OUT_PREFIX}_C_trimesh.obj")


'# https://github.com/mikedh/trimesh\nv 197.06622464 101.29328394 159.96470057\nv 255.86009088 253.83930695 171.27646596\nv 298.72129024 224.87893258 197.70071491\nv 168.92724480 136.00084041 120.87784468\nv 275.13704192 233.72147393 79.84609912\nv 279.10663680 145.57538872 129.73221426\nv 300.95506176 169.58351293 156.89798351\nv 290.64756224 252.66794794 125.80172060\nv 267.28864001 254.22716757 170.32062005\nv 252.71020289 172.53666235 72.95045108\nv 309.96795649 181.46621500 219.42580103\nv 258.80011777 219.31023740 220.73882523\nv 303.75540481 203.88592807 226.50191528\nv 153.46959106 128.75761117 186.18204395\nv 172.11460866 194.86962679 73.79917140\nv 220.85050626 239.74038961 184.71590822\nv 264.91038466 169.21997646 79.09671581\nv 253.60746242 198.47449237 255.17534895\nv 242.94346498 142.90397250 82.93127544\nv 288.02447874 161.91000971 139.32641808\nv 301.42922754 181.90656653 263.78359894\nv 191.03786242 187.01618422 113.33472869\nv 213.37057026 173.89855937 142.63318545\nv

In [35]:
# --- SUPER-SMOOTH LIVER MESH (VTK-free) ---
import numpy as np
import nibabel as nib
from scipy.ndimage import distance_transform_edt, gaussian_filter, zoom, map_coordinates
from skimage.restoration import denoise_tv_chambolle
from skimage import measure
import open3d as o3d
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --------- CONFIG ---------
# NIFTI_PATH = "liver_mask_resampled_2mm.nii.gz"  # <-- change me
NIFTI_PATH = '/home/ralbe/pyhppc_project/cirr_segm_clean/cirrhotic_liver_segmentation/cirr_mri_600/data/processed/cirrhotic/T1_masks/GT/482.nii.gz'
TARGET_VERTS = 2500
# Aggressive but volume-safe defaults:
UP_ISO_MM = 0.8           # extract on finer SDF grid
TV_WEIGHT = 0.06          # SDF TV denoise strength
TV_ITERS = 120
GAUSS_SIGMA_MM = 2.0      # SDF Gaussian sigma in mm
TAUBIN_ITERS = 80
BILAT_ITERS = 3
BILAT_SIGMA_LEN = 1.8     # ~mm on mesh
BILAT_SIGMA_ANG = 0.14
SAVE_PLY = True
PLY_OUT = "liver_super_smooth.ply"


# --- Strong, safe smoothing without bilateral (Open3D + optional trimesh HC) ---
import numpy as np
import open3d as o3d

def smooth_mesh_strong(V, F,
                       laplacian_pre=10,          # light pre-smoothing
                       taubin_iters=120,          # strong Taubin
                       taubin_lambda=0.5, taubin_mu=-0.53,
                       try_trimesh_hc=True, hc_iters=10, hc_alpha=0.1, hc_beta=0.5):
    """
    Returns smoothed (V, F). Uses Open3D Laplacian + Taubin.
    If trimesh is available, applies optional Humphrey/HC smoothing for extra de-rippling.
    """
    m = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(V),
        triangles=o3d.utility.Vector3iVector(F)
    )
    # Clean first (avoid NaNs during smoothing)
    m.remove_duplicated_vertices()
    m.remove_degenerate_triangles()
    m.remove_duplicated_triangles()
    m.remove_non_manifold_edges()
    m.remove_unreferenced_vertices()  # important before smoothing
    # Open3D smoothing: Laplacian (light) + Taubin (volume-friendly)
    if laplacian_pre > 0:
        m = m.filter_smooth_laplacian(number_of_iterations=laplacian_pre, lambda_filter=0.5)
    m = m.filter_smooth_taubin(number_of_iterations=taubin_iters,
                               lambda_filter=taubin_lambda, mu=taubin_mu)
    m.compute_vertex_normals()

    V_out = np.asarray(m.vertices).copy()
    F_out = np.asarray(m.triangles).astype(np.int32).copy()

    # Optional HC smoothing (trimesh), if installed
    if try_trimesh_hc:
        try:
            import trimesh
            tm = trimesh.Trimesh(vertices=V_out, faces=F_out, process=False)
            trimesh.smoothing.filter_humphrey(tm, alpha=hc_alpha, beta=hc_beta, iterations=hc_iters)
            V_out = tm.vertices.copy()
            F_out = tm.faces.astype(np.int32).copy()
        except Exception:
            pass  # silently skip if trimesh isn't present

    return V_out, F_out





# ---------- I/O ----------
def load_nifti_mask(path):
    nii = nib.load(path)
    vol = nii.get_fdata()
    mask = (vol > 0).astype(np.uint8)
    spacing = tuple(float(z) for z in nii.header.get_zooms()[:3])  # (sx, sy, sz)
    return mask, spacing

# ---------- Field ops ----------
def compute_sdf_mm(mask, spacing):
    dt_in  = distance_transform_edt(mask,     sampling=spacing)
    dt_out = distance_transform_edt(1 - mask, sampling=spacing)
    return dt_in - dt_out  # +inside / -outside, in mm

def upsample_sdf(sdf, spacing, new_iso):
    # sdf is (z,y,x); spacing is (sx,sy,sz) but zoom needs factors in z,y,x
    sx, sy, sz = spacing
    zoom_factors = (sz / new_iso, sy / new_iso, sx / new_iso)
    sdf_up = zoom(sdf, zoom=zoom_factors, order=1)  # trilinear
    return sdf_up, (new_iso, new_iso, new_iso)

def voxel_volume_mm3(spc):
    return float(spc[0] * spc[1] * spc[2])

def mask_volume_mm3(mask, spacing):
    return float(mask.sum()) * voxel_volume_mm3(spacing)

def volume_from_sdf_level(sdf, spacing, level):
    return float((sdf >= level).sum()) * voxel_volume_mm3(spacing)

def find_volume_matched_isolevel(sdf, spacing, target_vol_mm3, tol_rel=0.01, max_iter=40):
    # Try to bracket around 0
    lo, hi = -10.0 * spacing[0], 10.0 * spacing[0]
    vol_lo = volume_from_sdf_level(sdf, spacing, lo)
    vol_hi = volume_from_sdf_level(sdf, spacing, hi)
    if not (vol_lo >= target_vol_mm3 >= vol_hi):
        # fallback: broaden to min/max
        lo = float(max(lo, sdf.min()))
        hi = float(min(hi, sdf.max()))
        vol_lo = volume_from_sdf_level(sdf, spacing, lo)
        vol_hi = volume_from_sdf_level(sdf, spacing, hi)
        if not (vol_lo >= target_vol_mm3 >= vol_hi):
            return 0.0  # safe fallback

    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        vol_mid = volume_from_sdf_level(sdf, spacing, mid)
        err = (vol_mid - target_vol_mm3) / max(1e-9, target_vol_mm3)
        if abs(err) <= tol_rel:
            return mid
        if vol_mid > target_vol_mm3:
            lo = mid  # too big → raise level
        else:
            hi = mid
    return 0.5 * (lo + hi)

def mc_from_volume_iso(volume, spacing, level):
    # volume is (z,y,x); spacing is (sx,sy,sz) → skimage wants (sz,sy,sx)
    verts, faces, _, _ = measure.marching_cubes(volume, level=level, spacing=spacing[::-1])
    return verts[:, ::-1], faces.astype(np.int32)  # (x,y,z), faces

# ---------- Mesh ops ----------
def o3d_clean(mesh):
    mesh.remove_duplicated_vertices()
    mesh.remove_degenerate_triangles()
    mesh.remove_duplicated_triangles()
    mesh.remove_non_manifold_edges()
    mesh.compute_vertex_normals()
    return mesh

def smooth_mesh_taubin_bilateral(V, F, iters_taubin, iters_bilat, sigma_len, sigma_ang):
    m = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(V),
        triangles=o3d.utility.Vector3iVector(F)
    )
    m = m.filter_smooth_taubin(number_of_iterations=iters_taubin)
    for _ in range(max(0, iters_bilat)):
        m = m.filter_smooth_bilateral(number_of_iterations=1,
                                      sigma_length=sigma_len,
                                      sigma_angle=sigma_ang)
    return o3d_clean(m)

def decimate_to_target(mesh, target_verts):
    n = np.asarray(mesh.vertices).shape[0]
    if n > target_verts:
        target_tris = max(200, int(target_verts * 2))  # heuristic
        mesh = mesh.simplify_quadric_decimation(target_tris)
        mesh = o3d_clean(mesh)
    return mesh

# ---------- Plotly helpers ----------
def mesh_edges_wireframe(V, F):
    edges = set()
    for t in F:
        i, j, k = map(int, t)
        for a, b in ((i,j),(j,k),(k,i)):
            if a > b: a, b = b, a
            edges.add((a,b))
    X= []; Y= []; Z= []
    for a,b in edges:
        xa,ya,za = V[a]; xb,yb,zb = V[b]
        X += [xa, xb, None]; Y += [ya, yb, None]; Z += [za, zb, None]
    return X,Y,Z

def mask_isosurface_trace(mask, spacing, name="Mask"):
    nz, ny, nx = mask.shape
    x = np.arange(nx) * spacing[0]
    y = np.arange(ny) * spacing[1]
    z = np.arange(nz) * spacing[2]
    X, Y, Z = np.meshgrid(x,y,z,indexing='xy')
    vals = mask.transpose(2,1,0)
    return go.Isosurface(
        x=X.flatten(order='F'), y=Y.flatten(order='F'), z=Z.flatten(order='F'),
        value=vals.flatten(order='F'),
        isomin=0.5, isomax=0.5, surface_count=1, opacity=0.35, showscale=False,
        caps=dict(x_show=False, y_show=False, z_show=False),
        colorscale=[[0, "rgba(220,60,60,0.4)"], [1, "rgba(220,60,60,0.4)"]],
        name=name
    )

def wireframe_trace(V, F, name, color="#1f77b4"):
    x,y,z = mesh_edges_wireframe(V,F)
    return go.Scatter3d(x=x,y=y,z=z,mode="lines",
                        line=dict(color=color,width=1.0),name=name,showlegend=True)

# ---------- MAIN: make super-smooth mesh ----------
mask, spacing = load_nifti_mask(NIFTI_PATH)
print("Loaded mask:", mask.shape, "spacing (mm):", spacing)
orig_vol = mask_volume_mm3(mask, spacing)
print(f"Original mask volume: {orig_vol/1000:.2f} cm³")

# SDF (mm) on original grid → TV denoise → Gaussian in mm
sdf = compute_sdf_mm(mask, spacing)
try:
    sdf_tv = denoise_tv_chambolle(sdf, weight=TV_WEIGHT, channel_axis=None, max_num_iter=TV_ITERS)
except TypeError:  # older scikit-image
    sdf_tv = denoise_tv_chambolle(sdf, weight=TV_WEIGHT, multichannel=False, n_iter_max=TV_ITERS)

sig_vox = tuple(GAUSS_SIGMA_MM / s for s in spacing)
sdf_f = gaussian_filter(sdf_tv, sigma=sig_vox, mode='nearest')

# Up-sample SDF just for extraction
sdf_up, spc_up = upsample_sdf(sdf_f, spacing, new_iso=UP_ISO_MM)

# Volume-matched iso-level (so strong smoothing doesn't shrink)
iso_level = find_volume_matched_isolevel(sdf_up, spc_up, orig_vol, tol_rel=0.01)
print("Volume-matched iso-level:", iso_level)

# Extract surface and smooth
V, F = mc_from_volume_iso(sdf_up, spc_up, level=iso_level)
mesh = smooth_mesh_strong(V, F, TAUBIN_ITERS, BILAT_ITERS, BILAT_SIGMA_LEN, BILAT_SIGMA_ANG)
mesh = decimate_to_target(mesh, TARGET_VERTS)

V_final = np.asarray(mesh.vertices); F_final = np.asarray(mesh.triangles).astype(np.int32)
print("Final vertices:", V_final.shape[0], "triangles:", F_final.shape[0])

# Save
if SAVE_PLY:
    o3d.io.write_triangle_mesh(PLY_OUT, mesh, print_progress=False)
    print("Saved:", PLY_OUT)

# ---------- VISUALIZE ----------
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'scene'}, {'type':'scene'}]],
    subplot_titles=("Original Mask (isosurface)", "Super-smooth Mesh (wireframe)"))

fig.add_trace(mask_isosurface_trace(mask, spacing, "Mask"), row=1, col=1)
fig.add_trace(wireframe_trace(V_final, F_final, "Smooth mesh", color="#1f77b4"), row=1, col=2)

for i in range(1,3):
    scene_id = f"scene{i}" if i>1 else "scene"
    fig.layout[scene_id].update(
        xaxis=dict(title="X (mm)"), yaxis=dict(title="Y (mm)"), zaxis=dict(title="Z (mm)"),
        aspectmode="data", dragmode="orbit"
    )
fig.update_layout(height=650, width=1300, title="Staircase-free Liver Surface")
fig.show()

Loaded mask: (192, 192, 192) spacing (mm): (2.0, 2.0, 2.0)
Original mask volume: 1626.50 cm³


/tmp/ipykernel_1437853/1166088872.py:214: FutureWarning: `multichannel` is a deprecated argument name for `denoise_tv_chambolle`. It will be removed in version 1.0.Please use `channel_axis` instead.
  sdf_tv = denoise_tv_chambolle(sdf, weight=TV_WEIGHT, multichannel=False, n_iter_max=TV_ITERS)


Volume-matched iso-level: 0.0


AttributeError: 'tuple' object has no attribute 'vertices'

In [37]:
import pymeshlab as pms
import numpy as np

def pymeshlab_bilateral(V, F, steps=3, sigma_geom=2.0, sigma_photo=0.1):
    ms = pms.MeshSet()
    ms.add_mesh(pms.Mesh(V, F))
    # Bilateral smoothing (one of MeshLab’s filters). Parameter names may vary by version.
    ms.apply_filter('meshing_surface_smooth_bilateral',
                    steps=steps,
                    sigma_geom=sigma_geom,
                    sigma_photo=sigma_photo)
    m = ms.current_mesh()
    Vb = np.asarray(m.vertex_matrix(), dtype=float)
    Fb = np.asarray(m.face_matrix(), dtype=np.int32)
    return Vb, Fb

ModuleNotFoundError: No module named 'pymeshlab'